In [24]:
import pandas as pd
import json
# import spacy 
import copy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/jiayili/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jiayili/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# 处理数据

## 删除不合理数据项

In [28]:
## nq

#处理验证集
nq_validation = pd.read_csv('raw_data_nq-validation.tsv', sep='\t',header=None)
#删除数据中空白以及包含多个单词的数据
nq_validation = nq_validation.drop(nq_validation[(nq_validation[1].str.isspace())|(nq_validation[1].str.find(' ')>=0)].index)

#处理训练集
nq_train = pd.read_csv('raw_data_nq-train.tsv', sep='\t',header=None)
#删除数据中空白以及包含多个单词的数据
nq_train = nq_train.drop(nq_train[(nq_train[1].str.isspace())|(nq_train[1].str.find(' ')>=0)].index)

print("NQ训练集样本总数：",len(nq_train))
print("NQ验证集样本总数：",len(nq_validation))
nq_train.head()

NQ训练集样本总数： 22880
NQ验证集样本总数： 512


,0,1
14,how many players on a box lacrosse team?,six
15,when did the nba 3 second rule start?,1936
20,bridge on the river kwai fact or fiction?,fiction
26,are pure metals made of atoms or ions?,atoms
30,what year did the movie deuces come out?,2017


In [29]:
## webquestion
rows = []
with open('raw_webquestion_val.json', 'r') as f:
    webquestion_val = json.load(f)
for i in range(len(webquestion_val)):
    rows.append([webquestion_val[i]['qText'],webquestion_val[i]['answers'][0]])
webquestion_val = pd.DataFrame(rows)
webquestion_val = webquestion_val.drop(webquestion_val[(webquestion_val[1].str.isspace())|(webquestion_val[1].str.find(' ')>=0)].index)

rows = []
with open('raw_webquestion_trainmodel.json', 'r') as f:
    webquestion_trainmodel = json.load(f)
for i in range(len(webquestion_trainmodel)):
    rows.append([webquestion_trainmodel[i]['qText'],webquestion_trainmodel[i]['answers'][0]])
webquestion_trainmodel = pd.DataFrame(rows)
webquestion_trainmodel = webquestion_trainmodel.drop(webquestion_trainmodel[(webquestion_trainmodel[1].str.isspace())|(webquestion_trainmodel[1].str.find(' ')>=0)].index)

print("WebQuestion训练集样本总数：",len(webquestion_trainmodel))
print("WebQuestion验证集样本总数：",len(webquestion_val))
webquestion_trainmodel.head()

WebQuestion训练集样本总数： 858
WebQuestion验证集样本总数： 209


,0,1
2,what country is the grand bahama island in?,Bahamas
7,what two countries invaded poland in the begin...,Germany
10,which countries border the us?,Canada
11,what is nina dobrev nationality?,Bulgaria
12,what country does iceland belong to?,Iceland


## 使用mask替换主体

In [30]:
copydf = copy.deepcopy(nq_train)
copydf.head()

,0,1
14,how many players on a box lacrosse team?,six
15,when did the nba 3 second rule start?,1936
20,bridge on the river kwai fact or fiction?,fiction
26,are pure metals made of atoms or ions?,atoms
30,what year did the movie deuces come out?,2017


In [31]:
be_list = ['is','was','am', 'is', 'are','was','were','be']
do_list = ['do','does','did','done','doing']
def modify(inputStr,answerStr):
    
    inputStr = inputStr.strip().lower()
#     if(type(answerStr)=='float'):
    answerStr = str(answerStr).strip().lower()
    if(inputStr[-1]=='?'):
        inputStr = inputStr[:-1]
    tokens = nltk.word_tokenize(inputStr)
    tagged = nltk.pos_tag(tokens)
    
    result = ''
    token = ''

#     print(tagged)
    #处理并删除起始疑问词
    if(tagged[0][0]=='when'):
        token = 'when'
        tagged.pop(0)
        tagged.append(('in','IN'))
    elif(tagged[0][0]=='where'):
        token = 'where'
        tagged.pop(0)
        tagged.append(('in','IN'))
    elif((tagged[0][0] == 'who')|(tagged[0][0] == 'whom')):
        tagged.pop(0)
        tagged.insert(0,('[MASK]','NN'))
        who.append([' '.join([t[0] for t in tagged])+'.',answerStr])
        return ' '.join([t[0] for t in tagged])+'.'
    elif(tagged[0][0] == 'whose'):
        tagged.pop(0)
        tagged.insert(0,('[MASK]\'s','NN'))
        whose.append([' '.join([t[0] for t in tagged])+'.',answerStr])
        return ' '.join([t[0] for t in tagged])+'.'
    elif(tagged[0][0] == 'why'):
        token = 'why'
        tagged.pop(0)
        tagged.append(('because','IN'))
        tagged.append(('of','IN'))
    elif(tagged[0][0] == 'which'):
        token = 'which'
        tagged.pop(0)
    elif(tagged[0][0] == 'what'):
        token = 'what'
        tagged.pop(0)
    elif(tagged[0][0] == 'how'):
        token = 'how'
        tagged.pop(0)
        tagged.insert(len(tagged),('by','IN'))
    else:
        tagged.append(('[MASK]','NN'))
        result = ' '.join([t[0] for t in tagged])+'.'
        others.append([result,answerStr])
        return
        
    for index in range(len(tagged)):
        if(tagged[index][0] in be_list):
            verb_index = index
            verb_word = tagged[index]
            tagged = tagged[verb_index+1:]
            #判断最后是不是介词
            if((len(tagged)>=1)&(tagged[len(tagged)-1][1] not in ['IN','TO'])):
                tagged.append(verb_word)
            tagged.append(('[MASK]','NN'))
            result = ' '.join([t[0] for t in tagged])+'.'
            break
        if((tagged[index][0] in do_list)|(tagged[index][1]=='MD')):
            verb_index = index
            verb_word = tagged[index][0]
            tagged = tagged[verb_index+1:]
            tagged.append(('[MASK]','NN'))
            result = ' '.join([t[0] for t in tagged])+'.'
            break
            
    if(len(result)==0):
        tagged.append(('[MASK]','NN'))
        result = ' '.join([t[0] for t in tagged])+'.'
        
    if(token == 'how'):
        how.append([result,answerStr])
    elif(token == 'what'):
        what.append([result,answerStr])
    elif(token == 'which'):
        which.append([result,answerStr])
    elif(token == 'where'):
        where.append([result,answerStr])
    elif(token == 'when'):
        when.append([result,answerStr])
    elif(token == 'why'):
        why.append([result,answerStr])
    return result

In [32]:
where = []
when = []
what = []
which = []
# what_ = []
who = []
whose = []
why = []
how = []
# how_ = []
others = []
for index, row in copydf.iterrows():
    row[0] = modify(row[0],row[1])

In [33]:
##测试问句转换陈述句效果
question_one = 'what has been offered to the poor homeless children for the coming winter?'
question_two = 'what movies does taylor lautner play in?'
question_three = 'how much is the fish?'

q1 = modify(question_one,'')
print(q1)
q2 = modify(question_two,'')
print(q2)
q3 = modify(question_three,'')
print(q3)

has been offered to the poor homeless children for the coming winter [MASK].
taylor lautner play in [MASK].
the fish by [MASK].


In [354]:
question_list = [where,when,what,which,who,whose,why,how,others]
what
# for each in question_list:
#     print(len(each))

[['the movie deuces come out [MASK].', '2017'],
 ['the name of the youth loved by salmacis is [MASK].', 'hermaphroditus'],
 ['the proper term used for the signature given by celebrities to their fans is [MASK].',
  'autograph'],
 ['the official religion of the soviet union was [MASK].', 'atheism'],
 ['religious groups lived in the new england colonies [MASK].', 'puritans'],
 ['our town by thornton wilder is [MASK].', 'drama'],
 ['they speak in beijing [MASK].', 'mandarin'],
 ['the zombies called in the last of us are [MASK].', 'infected'],
 ["'s the abv of pabst blue ribbon [MASK].", '4.6%'],
 ['you call a person who does acrobatics [MASK].', 'acrobat'],
 ['the name of the cat in tom and jerry is [MASK].', 'tom'],
 ['the capital of azerbaijan in europe is [MASK].', 'baku'],
 ['it called when a vampire hypnotizes you in vampire diaries is [MASK].',
  'compel'],
 ["gecko 's name on pj masks is [MASK].", 'greg'],
 ['the latest is [MASK].', '64.0'],
 ['the tallest mountain in the north ame

In [336]:
copydf[130:180]

,0,1
454,the money called in spain is [MASK].,Euro
455,the civil war in libya in [MASK].,2011
456,countries border lake victoria [MASK].,Kenya
457,thomas jefferson died [MASK].,Charlottesville
460,spanish springs nv in [MASK].,Europe
461,[MASK] won the 2012 snooker masters.,London
468,[MASK] was the first jedi master.,Yoda
471,thomas jefferson died in [MASK].,Charlottesville
472,ibrahim chatta from in [MASK].,Haryana
476,dr carter g woodson do [MASK].,Writer


In [322]:
# copydf.to_csv("new_webquestion_val.csv",index=False,sep='\t',header=None)

## 保存加工后数据

In [268]:
# #保存处理后的数据
# nq_validation.to_csv("data_nq-validation.csv",index=False,sep='\t',header=None)
# nq_train.to_csv("data_nq-train.csv",index=False,sep='\t',header=None)
# webquestion_val.to_csv("webquestion_val.csv",index=False,sep='\t',header=None)
# webquestion_trainmodel.to_csv("webquestion_trainmodel.csv",index=False,sep='\t',header=None)

## 测试数据

In [2]:
# !pip install transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
# M_after = pipeline(
#     "fill-mask",
#     model="/tmp/test-mlm",
# )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
unmasker("China's now capital is [MASK]!")

[{'sequence': "china's now capital is dead!",
  'score': 0.06666794419288635,
  'token': 2757,
  'token_str': 'dead'},
 {'sequence': "china's now capital is gone!",
  'score': 0.04279176890850067,
  'token': 2908,
  'token_str': 'gone'},
 {'sequence': "china's now capital is destroyed!",
  'score': 0.039119113236665726,
  'token': 3908,
  'token_str': 'destroyed'},
 {'sequence': "china's now capital is burning!",
  'score': 0.035776544362306595,
  'token': 5255,
  'token_str': 'burning'},
 {'sequence': "china's now capital is falling!",
  'score': 0.034721579402685165,
  'token': 4634,
  'token_str': 'falling'}]

In [291]:
# question_list = [where,when,what,which,who,whose,why,how,others]
memo = []
memo_2 = []
for each in question_list:
#     print(len(each))
    top5answers = []
    topanswer = []
    for question in each:
        print(question)
        res = unmasker(question[0])
        temp = []
        topanswer.append(res[0]['token_str'])
        for eachresult in res:
            temp.append(eachresult['token_str'])
        top5answers.append(temp)
    memo.append(topanswer)
    memo_2.append(top5answers)
# memo

['the ilium the ischium and the pubis meet in [MASK].', 'acetabulum']
['the first battle of the texas revolution take place in [MASK].', 'gonzales']
["the macy 's thanksgiving day parade held in [MASK].", 'manhattan']
['esperanza live in the house on mango street in [MASK].', 'chicago']
['national open university located in abuja in [MASK].', 'jabi']
['the masterchef kitchen located in melbourne 2017 in [MASK].', 'flemington']
['the high court of kerala located in [MASK].', 'kochi']
['the term dead end come from in [MASK].', 'french']
['the first home alone take place in [MASK].', 'chicago']
['andy go at the end of shawshank redemption in [MASK].', 'zihuatanejo']
['the french and indian war start in [MASK].', 'pennsylvania']
['the united kingdom come in the eurovision song contest in [MASK].', '24th']
['bile go after it leaves the gallbladder in [MASK].', 'duodenum']
['the cruise ships leave from miami in [MASK].', 'portmiami']
['chitty chitty bang bang car made in [MASK].', 'hertfords

['the family name price come from in [MASK].', 'welsh']
['the largest annual temperature range found in [MASK].', 'verkhoyansk']
['scooby doo 2 monsters unleashed filmed in [MASK].', 'vancouver']
["india 's economy rank in the world in [MASK].", 'sixth-largest']
['an eye for an eye western filmed in [MASK].', '1966']
["the vibranium for captain america 's shield come from in [MASK].", 'wakanda']
['the summer olympics be held in 2020 in [MASK].', 'tokyo']
['the 2024 olympics gon na be held in [MASK].', 'paris']
['the world cup be hosted next in [MASK].', 'qatar']
['the last name duran originate from in [MASK].', 'spanish']
['be the 2022 fifa world cup held in [MASK].', 'qatar']
['they find the boston marathon bombers in [MASK].', 'watertown']
['the money for subsidies come from in [MASK].', 'government']
['the mississippi river begin in which state in [MASK].', 'minnesota']
['so mote it be come from in [MASK].', 'england']
['the titanic make its maiden voyage from in [MASK].', 'southamp

["canada 's greatest concentration of fossil fuel resources located in [MASK].", 'alberta']
['eurovision 2018 going to be held in [MASK].', 'lisbon']
['the hemlock wooly adelgid come from in [MASK].', 'japan']
['the water from clothes go when they are dried in sun name this process in [MASK].', 'evaporation']
['the 2020 olympic games going to be in [MASK].', 'tokyo']
['the estate used in magnum pi in [MASK].', 'waimānalo']
['do start in the key of c in [MASK].', 'c']
['the olympic torch lit in greece in [MASK].', 'olympia']
['belgium come in the world cup in [MASK].', 'third']
['the hormones in the pill come from in [MASK].', 'steroid']
['the summer olympics going to be in 2 years in [MASK].', 'tokyo']
['euro 2024 going to be held in [MASK].', 'germany']
['the 2022 soccer world cup be held in [MASK].', 'qatar']
['the bridges of madison county take place in [MASK].', 'iowa']
['in the cold light of day filmed in [MASK].', 'spain']
['gobi desert located on the world map in [MASK].', 'asia

['russia rank in the world economy in [MASK].', '11']
["most of russia 's territory located in [MASK].", 'siberia']
['milkman live in song of solomon in [MASK].', 'michigan']
['the oil industry begin in the south in 1901 in [MASK].', 'texas']
['the crocodile episode of black mirror take place in [MASK].', 'iceland']
['liverpool finish in the league last season in [MASK].', '8']
['the term cupboard love come from in [MASK].', 'english']
['they film the assasination of gianni versace in [MASK].', 'miami']
['gb come in the 2012 olympics in [MASK].', '3rd']
["the world 's largest desert located in [MASK].", 'antarctica']
['water enter the vascular system of a starfish in [MASK].', 'madreporite']
['the land of steady habits movie filmed in [MASK].', 'tarrytown']
['the term saved by the bell originate in [MASK].', 'boxing']
['the world cup held in 2018 in [MASK].', 'russia']
['the head office of rbi located in [MASK].', 'mumbai']
['we find the highest genetic diversity among human population

['the first colonial settlements in australia founded in [MASK].', 'sydney']
['the ancient greece olympics take place in [MASK].', 'olympia']
['the last name ketchum come from in [MASK].', 'england']
['from dusk till dawn take place in [MASK].', 'mexico']
['the world cup be played in 2022 in [MASK].', 'qatar']
['the term hoi polloi come from in [MASK].', 'greek']
['the boy in striped pajamas take place in [MASK].', 'berlin']
['the movie cargo based out of in [MASK].', 'australia']
['the guy from into the wild die in [MASK].', 'starvation']
['the largest museum of india located in [MASK].', 'kolkata']
['diary of a wimpy kid the getaway take place in [MASK].', 'christmas']
['the show the ozarks take place in [MASK].', 'missouri']
['route 66 start on the east coast in [MASK].', 'chicago']
['the 2023 rugby world cup be held in [MASK].', 'france']
['the last name lucio come from in [MASK].', 'italian']
['the energy in a polysaccharide stored in [MASK].', 'glycogen']
['the seminole lived bef

...

['in what year did allstate go public when sears sells 19.8 percent of the company [MASK].', '1993']
['pro kabaddi 2017 how many matches per team [MASK].', '22']
['most goals scored by a premier league team in a season [MASK].', '106']
["hunt 's first name on grey 's anatomy [MASK].", 'owen']
['at what age can you get a tattoo in new york [MASK].', '18']
["the public works administration help make improvements to what part of the nation 's economy [MASK].", 'internal']
['new england clam chowder is that the red or the white [MASK].', 'white']
['age of tree by counting the annual ring [MASK].', 'dendrochronology']
['high speed train los angeles to las vegas [MASK].', 'xpresswest']
['at which stage does a silkworm produce silk thread [MASK].', 'larvae']
["it 's getting hot in here nelly year [MASK].", '2002']
['asian restaurant cook food in front of you [MASK].', 'teppanyaki']
['nightmare on elm street how many are there [MASK].', 'nine']
['the dancer figurine from mohenjo daro civilizat

["the influential novel uncle tom 's cabin was written in response to [MASK].", 'slavery']
['highest goal scored in a game in world cup [MASK].', '5']
['the effect of french revolution on english literature [MASK].', 'romanticism']
['the near eastern area that was known in the ancient world as persia is today called [MASK].', 'iran']
['28 days later disease spread from which primate [MASK].', 'chimpanzees']
['most goals scored in champions league game by one player [MASK].', '7']
['the two broad types of control structures is branching and [MASK].', 'linear']
['american female singer who died in a plane crash [MASK].', 'aaliyah']
['the enzyme in whose presence glucose and fructose are converted into alcohol [MASK].', 'yeast']
['the number of orders in class mammalia is [MASK].', '29']
['excessive amount of sugar in the blood is called [MASK].', 'hyperglycemia']
['according to the united nations in 2015 which country had the highest life expectancy at birth [MASK].', 'japan']
['a calcul

['rust on an iron nail element or compound [MASK].', 'compound']
['in the early 1600s what crime was used extensively by the english court of the star chamber [MASK].', 'sedition']
['mitochondria is found in what type of cells [MASK].', 'eukaryotic']
['ps i love you who wrote the letters [MASK].', 'gerry']
['a measure using algorithms to publically protect patient identifiable information is [MASK].', 'k-anonymization']
['never let me go what are they donating [MASK].', 'organs']
['the festival of holi marks the end of winter and the beginning of [MASK].', 'spring']
['an artwork behind the alter in some churches [MASK].', 'altarpiece']
['the song that has the most views on youtube [MASK].', 'despacito']
['amylase is a digestive enzyme that breaks down starch into sugars through [MASK].', 'digestion']
['third book of the old testament 9 letters [MASK].', 'leviticus']
['esq at the end of a name uk [MASK].', 'esquire']
['the source of authority cited by the supreme court for applying the 

['best country with english as a second language [MASK].', 'europe']
['sachin tendulkar highest score in one day cricket [MASK].', '200']
['name the type of blood vessels which carry blood from organs to the heart [MASK].', 'venous']
['once upon a time in india title of which film [MASK].', 'lagaan']
['number of episodes in season 5 of the 100 [MASK].', '13']
['fear the walking dead how many episodes season 3 [MASK].', '16']
['record number of goals scored in a world cup game [MASK].', '10']
['one of the first signs of the onset of tetanus is trismus also referred to as [MASK].', 'lockjaw']
['are black eyed peas a bean or vegetable [MASK].', 'bean']
['book about a woman who travels back in time to scotland [MASK].', 'outlander']
['one of the pointed divisions of a fork [MASK].', 'tines']
['the branch of zoology that deals with the study of fishes [MASK].', 'ichthyology']
['in the bible who got swallowed by a whale [MASK].', 'jonah']
['the number of layers in iso osi reference model [MA

['queen of the iceni who revolted against the romans codycross [MASK].', 'boudica']
['the conquered city of constantinople became the capital of which empire [MASK].', 'ottoman']
['the national honour award was established by nigerian government in the year [MASK].', '1964']
['indian railway network connects about how many stations [MASK].', '7,349']
['most losses in a row in nfl history [MASK].', '26']
['the word masjid ( mosque ) literally means a place of [MASK].', 'worship']
['meiosis ii does not take place in the oocyte until [MASK].', 'fertilization']
['in 1804 the french colony of achieved independence [MASK].', 'haiti']
['in which kingdom the bahubali series mainly set [MASK].', 'mahismati']


In [292]:
evalmemo = []
for i in range(len(memo)):
    correct = 0
    correct_in_5 = 0
    index = 0
    for answer in question_list[i]:
#         print(memo[i][index])
        if(answer[1] == memo[i][index]):
            correct=correct+1
        if(answer[1] in memo_2[i][index]):
            correct_in_5=correct_in_5+1
        index = index+1
    evalmemo.append([index,correct,correct_in_5])
    print(evalmemo[-1])
    

[1557, 111, 267]
[6754, 19, 76]
[3542, 143, 280]
[594, 21, 45]
[3802, 35, 352]
[2, 0, 0]
[14, 1, 1]
[3301, 0, 2]
[3314, 90, 195]


In [241]:
evalmemo
# print("数据总数共有:",index)
# print("回答在第一名的有",correct)
# print("回答在前五名的有",correct_in_5)

[[234, 39, 78]]

## 使用新模型测试

In [345]:
# question_list = [where,when,what,which,who,whose,why,how,others]
memo = []
memo_2 = []
for each in question_list:
#     print(len(each))
    top5answers = []
    topanswer = []
    for question in each:
        print(question)
        res = M_after(question[0])
        temp = []
        topanswer.append(res[0]['token_str'])
        for eachresult in res:
            temp.append(eachresult['token_str'])
        top5answers.append(temp)
    memo.append(topanswer)
    memo_2.append(top5answers)
# memo

['rome italy located on a map in [MASK].', 'rome']
['france colonise in [MASK].', 'réunion']
['to visit in n. ireland in [MASK].', 'antrim']
['kennedy when he got shot in [MASK].', 'dallas']
['the massive allied invasion of northern france take place in june 1944 in [MASK].', 'normandy']
['mitsubishi headquarters in [MASK].', 'tokyo']
['the australian prime minister reside in [MASK].', 'canberra']
['abraham lincoln hometown in [MASK].', 'hodgenville']
['zachary taylor elected from in [MASK].', 'kentucky']
['puntland somalia in [MASK].', 'garoowe']
['hackney 2012 held in [MASK].', 'london']
['bruno mars get his start in [MASK].', 'honolulu']
['the sultan of the ottoman empire live in [MASK].', 'constantinople']
['audi come from in [MASK].', 'ingolstadt']
['morgan freeman come from in [MASK].', 'memphis']
['eu headquarters located in [MASK].', 'brussels']
['john boehner come from in [MASK].', 'ohio']
['giovanni da verrazano come from in [MASK].', 'italy']
['martin luther live in in [MASK

['the major religions in poland are [MASK].', 'catholicism']
['country volvo cars made [MASK].', 'sweden']
['anderson cooper do [MASK].', 'journalist']
["howard carter 's job was [MASK].", 'archaeologist']
['antonio vivaldi known for [MASK].', 'priest']
['movie queen latifah play in [MASK].', 'taxi']
['toronto canada located in [MASK].', 'ontario']
['japan export to [MASK].', 'canada']
['virginia in [MASK].', 'utc-5']
['chadwick discover [MASK].', 'neutron']
['germany share borders with [MASK].', 'netherlands']
['the cause of death for edgar allan poe was [MASK].', 'tuberculosis']
['the capital city of west germany is [MASK].', 'bonn']
['used in germany 2012 is [MASK].', 'euro']
['marco polo come from [MASK].', 'venice']
['south america made up of [MASK].', 'carrasco']
['movies have gerard butler been in [MASK].', 'attila']
['present day countries use english as their national language [MASK].', 'canada']
['movies gerard butler in [MASK].', 'attila']
['liam hemsworth in [MASK].', 'know

In [346]:
evalmemo = []
for i in range(len(memo)):
    correct = 0
    correct_in_5 = 0
    index = 0
    for answer in question_list[i]:
#         print(memo[i][index])
        if(answer[1] == memo[i][index]):
            correct=correct+1
        if(answer[1] in memo_2[i][index]):
            correct_in_5=correct_in_5+1
        index = index+1
    evalmemo.append([index,correct,correct_in_5])
    print(evalmemo[-1])

[77, 13, 18]
[8, 0, 0]
[98, 7, 16]
[10, 1, 1]
[15, 0, 0]
[0, 0, 0]
[0, 0, 0]
[1, 0, 0]
[0, 0, 0]
